# Topological clustering analysis - information maps

Self-organised feature maps in the final layer representing specific convex boundary elements after network training.

This plots Fig 10 and supplementary S2 Fig.

**Dependencies**

Note that if an inference recording has already been saved to disk, then the workflow skips this file.

- `./scripts/run_main_workflow.py experiments/n3p2/train_n3p2_lrate_0_04_181023 31 --chkpt -1 --rule inference -v`
- `./scripts/run_main_workflow.py experiments/n4p2/train_n4p2_lrate_0_02_181023 15 --chkpt -1 --rule inference -v`

**Plots**

- N3P2 feature map (convex- or concave-selective neurons)
- N4P2 feature map (convex- or concave-selective neurons)

In [ ]:
from enum import Enum

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib.ticker import MaxNLocator
from pydantic import BaseModel
from tqdm.notebook import tqdm

from hsnn import analysis, utils, viz
from hsnn.analysis import measures
from hsnn.utils import handler, io
from hsnn.utils.data import ImageSet
from hsnn.utils.handler import TrialView

OUTPUT_DIR = io.BASE_DIR / "out/figures/fig10"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


class DataSet(Enum):
    N3P2 = 1
    N4P2 = 2


def load_inference_results(
    trial: TrialView, chkpt: int | None, **kwargs
) -> xr.DataArray:
    results_path = handler.get_results_path(trial, chkpt, **kwargs)
    if results_path.is_file():
        print(f"Loading '{results_path}'...")
        return utils.io.load_pickle(results_path)
    else:
        raise FileNotFoundError(f"'{results_path}'")


class InferenceConfig(BaseModel):
    """Inference kwargs passed to `handler.load_results`."""

    amplitude: int
    subdir: str | None = None


viz.setup_journal_env()

### 1) Load N3P2 and N4P2 recorded results

**Get representative Trial per (`E2E`, `FF`) combination**

In [ ]:
# Experiment data structures
logdirs = {
    DataSet.N3P2: "n3p2/train_n3p2_lrate_0_04_181023",
    DataSet.N4P2: "n4p2/train_n4p2_lrate_0_02_181023",
}
imagesets: dict[DataSet, tuple[ImageSet, pd.DataFrame]] = {}

# Trial data structures
dataset_trial_mapping = {
    DataSet.N3P2: (20, 20, 7),
    DataSet.N4P2: (20, 20, 3),
}
state_chkpt_mapping = {"post": -1}
records_ds: dict[DataSet, dict[str, xr.DataArray]] = {
    dataset: {"post": None} for dataset in DataSet
}

inference_cfg = InferenceConfig(amplitude=0, subdir=None)
offset = 0.0 if inference_cfg.subdir == "onsets" else 50.0

In [ ]:
for dataset, logdir in logdirs.items():
    expt = handler.ExperimentHandler(logdir)
    dataset_name = expt.logdir.parent.stem
    trial = expt[dataset_trial_mapping[dataset]]
    # Recordings per state
    for state, chkpt in state_chkpt_mapping.items():
        records_ds[dataset][state] = load_inference_results(
            trial, chkpt, **dict(inference_cfg)
        )
    # Imagesets
    cfg = trial.config
    if inference_cfg.amplitude > 0:
        cfg["training"]["data"]["transforms"]["gaussiannoise"] = [
            inference_cfg.amplitude
        ]
    imagesets[dataset] = utils.io.get_dataset(
        cfg["training"]["data"], return_annotations=True
    )
else:
    # Common parameters
    duration: float = (
        records_ds[dataset]["post"].item(0).duration - offset
    )  # Observation period
    reps = len(records_ds[dataset]["post"]["rep"])
    input_shape = tuple(cfg["topology"]["poisson"]["EXC"])
    layer_shape = tuple(cfg["topology"]["spatial"]["EXC"])

### 2) Plot information maps for each dataset

First get the coordinates of informative neurons

Requires pip package: `alphashape`

In [ ]:
import alphashape
from scipy.interpolate import splev, splprep
from shapely.geometry import MultiPolygon, Polygon


def get_coordinates_values(
    records: xr.DataArray,
    labels: pd.DataFrame,
    threshold: float,
    duration: float,
    offset: float,
    layer: int = 4,
    target: int = 1,
) -> tuple[dict[str, np.ndarray], dict[str, np.ndarray]]:
    """Gets the coordinates of informative neurons, along with their associated specific information
    measures, for each side of objects annotated by labels. Coordinates are given as [(x, y), ...].

    Returns:
        tuple[dict[str, np.ndarray], dict[str, np.ndarray]]: coordinates and associated specific measures.
    """
    rates_array = analysis.infer_rates(
        records.sel(layer=layer, nrn_cls="EXC"), duration, offset
    )
    sides = list(labels.columns[1:])

    coordinates: dict[str, np.ndarray] = {}
    values: dict[str, np.ndarray] = {}
    for side in sides:
        specific_measures = measures.get_sorted_measures_rates(
            rates_array, labels, side, target
        )
        indices = specific_measures[specific_measures["measure"] > threshold].index
        values[side] = np.asarray(specific_measures.loc[indices]["measure"])
        _coords = []
        for i, index in enumerate(indices):
            y, x = np.unravel_index(index, layer_shape)
            _coords.append((x, y))
        coordinates[side] = np.array(_coords)
    return coordinates, values


# Function to plot the smoothed alpha shapes of clusters with borders only
def plot_smooth_alpha_shapes(axes, points, color, alpha=0.5):
    if len(points) >= 3:  # AlphaShape requires at least 3 points
        alpha_shape = alphashape.alphashape(points, alpha)
        if isinstance(
            alpha_shape, (Polygon, MultiPolygon)
        ):  # Check if the alpha shape is a valid polygon or multipolygon
            if isinstance(alpha_shape, Polygon):
                smooth_plot(axes, alpha_shape.exterior.xy, color)
                # add_annotation(axes, alpha_shape.centroid, side)
            else:
                for geom in alpha_shape.geoms:
                    smooth_plot(axes, geom.exterior.xy, color)
                    # add_annotation(axes, geom.centroid, side)


# Function to plot the smoothed polygon with borders only
def smooth_plot(axes, xy, color):
    x, y = xy
    tck, u = splprep([x, y], s=0.0, per=True)
    u_new = np.linspace(u.min(), u.max(), 1000)
    x_new, y_new = splev(u_new, tck)
    axes.fill(
        x_new, y_new, color=color, alpha=0.7
    )  # Use plot instead of fill for borders only
    axes.plot(
        x_new, y_new, color="black", linestyle="-", linewidth=0.5
    )  # Use a solid black line for borders


# Function to add annotations
def add_annotation(axes, centroid, side):
    axes.text(
        centroid.x,
        centroid.y,
        side,
        fontsize=12,
        ha="center",
        va="center",
        color="black",
    )

In [ ]:
# Common setup
state = "post"
layer = 4
threshold = 2 / 3
target = 0  # Set target=1 for convex, or target=0 for concave

coordinates_ds: dict[DataSet, dict[str, np.ndarray]] = {}
values_ds: dict[DataSet, dict[str, np.ndarray]] = {}

**Get informative neurons per dataset**

In [ ]:
for dataset in tqdm(DataSet):
    records = records_ds[dataset][state]
    _, labels = imagesets[dataset]

    coordinates_ds[dataset], values_ds[dataset] = get_coordinates_values(
        records, labels, threshold, duration, offset, layer, target
    )

**Plot information maps**

In [ ]:
f, axes_grid = plt.subplots(1, 2, figsize=(5.5, 3), sharex=False, sharey=False)

for i, dataset in enumerate(DataSet):
    axes = axes_grid[i]
    _, labels = imagesets[dataset]
    sides = list(labels.columns[1:])

    # Plot information map
    color_cycler = plt.rcParams["axes.prop_cycle"]
    colors = [elem["color"] for elem in color_cycler]

    # Set the background color to a specific grayscale value
    axes.set_facecolor("0.7")

    for idx, side in enumerate(sides):
        points = np.array(coordinates_ds[dataset][side])
        plot_smooth_alpha_shapes(
            axes, points, color=colors[idx], alpha=(0.15 if target == 0 else 0.2)
        )

    axes.xaxis.set_major_locator(MaxNLocator(nbins=4))
    axes.yaxis.set_major_locator(MaxNLocator(nbins=4))
    axes.set_xlabel("X")
    if i == 0:
        axes.set_ylabel("Y", rotation=0, labelpad=10)
    axes.invert_yaxis()
    axes.set_title(f"Information map: {dataset.name}", size="large", fontweight="bold")
f.tight_layout()

fname = (
    "fig_information_maps_convex.pdf"
    if target == 1
    else "fig_information_maps_concave.pdf"
)
viz.save_figure(f, OUTPUT_DIR / fname, overwrite=False)

In [ ]:
# Plot individual neuron coordinates per side as a separate colour
f, axes = plt.subplots(figsize=(4, 4))
axes: plt.Axes
for side, coords in coordinates_ds[dataset].items():
    axes.scatter(coords[:, 0], coords[:, 1])
axes.invert_yaxis()